In [1]:
import os 
import yfinance as yf
import plotly.graph_objects as go
from agno.agent import Agent
from agno.models.google import Gemini 


In [ ]:
os.environ["GOOGLE_API_KEY"] = ""

In [2]:
def compare_stocks(symbols):
    data = {}
    for symbol in symbols:
        stock = yf.Ticker(symbol)
        hist = stock.history(period="1y")

        if hist.empty:
            print(f"No data found for {symbol}.")
            continue
        data[symbol] = hist['Close'].pct_change().sum()
    return data

In [3]:
#market analyst 
market_analyst = Agent(
    model = Gemini(id="gemini-1.5-flash"),
    description = "Market analyst",
    instructions=["Retrieve and compare stock performance from yahoo Finance.","Calculate pe3rcentage change over a 6 -month period.","Provide a summary of the stocks' performance."],
    show_tool_calls = True,
    markdown = True
)

In [4]:
#function to get market analysis 
def get_market_analysis(symbols):
    performance_data = compare_stocks(symbols)
    if not performance_data:
        return "No performance data available."
    analysis = market_analyst.run(f"Compare these stock performace : {performance_data}")
    return analysis.content

In [5]:
def get_company_info(symbol):
    stock = yf.Ticker(symbol)
    return {
        "name": stock.info.get("longName", "N/A"),
        "sector": stock.info.get("sector", "N/A"),
        "industry": stock.info.get("industry", "N/A"),
        "description": stock.info.get("longBusinessSummary", "N/A"),
    }

def get_company_news(symbol):
    stock = yf.Ticker(symbol)
    news = stock.news[:5]
    return news 




In [6]:
company_researcher = Agent(
    model =  Gemini(id="gemini-1.5-flash"),
    description = "Company researcher",
    instructions=["Retrieve company information and news from Yahoo Finance.","Summarise latest company news relevant to investor"],
    markdown = True
)

In [14]:
def get_company_analysis(symbol):
    company_info = get_company_info(symbol)
    news = get_company_news(symbol)
    response = company_researcher.run(
        f"Provide an analysis for {info['name'] }in the {info['sector']} sector.\n "
        f"Market Cap : {info['marketCap']} \n"
        f"Summary :  {info['summary']}\n"
        f"Latest news: {news} \n"
    )
    return response.content


In [11]:
stock_strategist = Agent(
    model = Gemini(id="gemini-1.5-flash"),
    description = "Stock strategist",
    instructions=["Provide stock recommendations based on market analysis and company research."],
    markdown = True
)

def get_stock_recommendations(symbols):
    market_analysis = get_market_analysis(symbols)
    data = {}
    for symbol  in stock_symbols:
        data[symbol] = get_company_analysis(symbol)
    recommendations = stock_strategist.run(
        f"Based on the market analysis : {market_analysis}, and company news {data}"
        f"Provide stock recommendations."
    )
    return recommendations.content

In [15]:
team_lead = Agent(
    model = Gemini(id="gemini-1.5-flash"),
    description = "Team lead who aggreagates the analysis and recommendations from the team.",
    instructions=["Summarise the analysis and recommendations from the team."],
    markdown = True

)

def get_final_investment_report(symbol):
    market_analysis = get_market_analysis(symbols)
    company_analysis = get_company_analysis(symbol)
    stock_recommendations = get_stock_recommendations(symbols)

    final_report = team_lead.run(
        f"Market Analysis: {market_analysis}\n"
        f"Company Analysis: {company_analysis}\n"
        f"Stock Recommendations: {stock_recommendations}"
    )
    return final_report.content

In [ ]:
input_symbol = input("Enter stock symbols separated by commas(AAPL,TSLA,GOOG): ")
stock_symbols = [symbol.strip() for symbol in input_symbol.split(",") if symbol.strip()]

In [ ]:
report = get_final_investment_report(stock_symbols)
print(report)